In [ ]:
import numpy as np
from time import time

In [ ]:
start_time = time()

In [ ]:
# Initialize parameters

n = 2048
NFFT = 100 # 274
Ndetect_far = 1775
Ndetect_near = 1125

epsconst = 0.8

Ny = n
Nx = n
Nmid = n // 2
dx = 100e-9  # in m
PML = 100
derror = 1e-4

a = 50 * dx
Nbox = 100

vf = 1500
vs = 1650

mu = 1.0
beta = 1.0
I0 = 1.0
Cp = 1.0

In [ ]:
r_dtype = "double"
c_dtype = "cdouble"

In [ ]:
kkxy = 2 * np.pi * np.arange(0, Nmid+1, dtype=r_dtype) / (Nx * dx)
f = kkxy * vf / (2 * np.pi)  # in Hz

f11 = f / 1000000
f

In [ ]:
dr = np.fromfunction(lambda i, j: (i - Nmid) ** 2 + (j - Nmid) ** 2, (Nx, Ny))

rg = np.sqrt(dr) * dx

cell_mask = rg <= a

In [ ]:
absorbing_layer = np.ones((Nx, Ny), dtype='bool')
absorbing_layer[PML:Nx-PML, PML:Ny-PML] = False

In [ ]:
# Calculate pmod2
pmod2 = dr * ((2 * np.pi / (Nx * dx)) ** 2)

In [ ]:
itern = []
time11 = []
error_CBS = []
PA_pressure_real_far = []
PA_pressure_imag_far = []
PA_pressure_abs_far = []
PA_pressure_real_near = []
PA_pressure_imag_near = []
PA_pressure_abs_near = []

In [ ]:
# Create empty matrices
V = np.zeros((Ny, Nx), dtype=c_dtype)
S = np.zeros((Ny, Nx), dtype=c_dtype)
Shimask = np.ones((Ny, Nx), dtype=r_dtype)

In [ ]:
# Main loop
for i1 in range(1, NFFT):
    iter_time = time()

    omega = 2 * np.pi * f[i1]
    ks = omega / vs
    kf = omega / vf
    epsilon = epsconst * kf * kf
    epsilon11 = np.sqrt(epsilon)

    V_in = ks * ks - kf * kf - 1j * epsilon
    V_out = -1j * epsilon
    V[:] = V_out
    V[cell_mask] = V_in

    S_in = -1j * omega * mu * beta * I0 / Cp
    S_out = 0
    S[:] = S_out
    S[cell_mask] = S_in

    # Calculate Shimask
    Shimask = np.exp(-epsilon11 * rg, dtype=r_dtype)
    Shimask[~absorbing_layer] = 1

    # Calculate G
    G = 1.0 / (pmod2 - kf * kf - 1j * epsilon)
    G = np.fft.fftshift(G)

    gamma = 1j * V / epsilon

    Shirin = gamma * np.fft.ifft2(G * np.fft.fft2(S))
    Shirfn = np.zeros((Nx, Ny), dtype=c_dtype)

    for i77 in range(2000):
        Shirfn = Shirin - (1j / epsilon) * V * (
            Shirin - np.fft.ifft2(G * np.fft.fft2(V * Shirin + S))
        )
        Shirfn = Shirfn * Shimask

        error_CBS2_CBS1 = np.linalg.norm(
            Shirfn[Nmid, :] - Shirin[Nmid, :], ord=1
        ) / np.linalg.norm(Shirin[Nmid, :], ord=1)

        if error_CBS2_CBS1 < derror:
            saturationCBS = i77
            break
        else:
            Shirin = Shirfn
        # if i77 % 20 == 0:
        # print("\r Inner:", i77, end="")

    itern.append(i77)
    time11.append(time() - iter_time)
    error_CBS.append(float(error_CBS2_CBS1))

    PA_pressure_real_far.append(float(np.real(Shirfn[Nmid, Ndetect_far])))
    PA_pressure_imag_far.append(float(np.imag(Shirfn[Nmid, Ndetect_far])))
    PA_pressure_abs_far.append(float(np.abs(Shirfn[Nmid, Ndetect_far])))

    PA_pressure_real_near.append(float(np.real(Shirfn[Nmid, Ndetect_near])))
    PA_pressure_imag_near.append(float(np.imag(Shirfn[Nmid, Ndetect_near])))
    PA_pressure_abs_near.append(float(np.abs(Shirfn[Nmid, Ndetect_near])))

    print(f"NFFT: {i1}, f: {f[i1]}, Saturation: {saturationCBS}, Time: {time() - iter_time}")

In [ ]:
total_time = time() - start_time
print("Total time:", total_time)

In [ ]:
# Save the results to files
FID11 = open("frequency_pomega_CBS_1950_far.txt", "w")
FID12 = open("frequency_pomega_CBS_1950_near.txt", "w")
FID22 = open("frequency_iteration_time_CBS_1950.txt", "w")

for i1 in range(NFFT):
    FID11.write(
        f"{f11[i1]},{PA_pressure_real_far[i1]},{PA_pressure_imag_far[i1]},{PA_pressure_abs_far[i1]}\n"
    )

    FID12.write(
        f"{f11[i1]},{PA_pressure_real_near[i1]},{PA_pressure_imag_near[i1]},{PA_pressure_abs_near[i1]}\n"
    )
    FID22.write(
        f"{f11[i1]},{itern[i1]},{error_CBS[i1]},{time11[i1]}\n"
    )

In [ ]:
# np.savetxt(
#     f"../result/Shirfn_{n}_real_cupy2.txt",
#     np.real(Shirfn),
#     delimiter=",",
# )
# np.savetxt(
#     f"../result/Shirfn_{n}_imag_cupy2.txt",
#     np.imag(Shirfn),
#     delimiter=",",
# )